In [17]:
#install 
#!sudo apt install tesseract-ocr
#!pip install easyocr
#!pip install opencv-python-headless==4.1.2.30
!pip install pdf2image
!apt-get install poppler-utils 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [38]:
import easyocr
import numpy as np
from pdf2image import convert_from_path
import cv2
from PIL import Image
import os
import PIL
from io import BytesIO

In [44]:
path='/content/sample_data/UntitledFolder'
fil=1
reader = easyocr.Reader(['en']) 
for i in os.listdir(path):
    print(i)
    print(i[-3:])
    if(i[-3:]=='PDF' or i[-3:]=='pdf'):
        #images = convert_from_path(path+'/'+i)
        #print(images)
        line=''
        image = np.concatenate(tuple(convert_from_path(path+'/'+i)), axis=0)
        #for j in range(len(images)):
            #image = cv2.imread(j)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Remove horizontal lines
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
        detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
        cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            cv2.drawContours(thresh, [c], -1, (0,0,0), 2)

        # Dilate to connect text and remove dots
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,1))
        dilate = cv2.dilate(thresh, kernel, iterations=2)
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 100:
                cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

        result = cv2.bitwise_and(image, image, mask=dilate)
        result[dilate==0] = (255,255,255)
        result1 = reader.readtext(result,detail=0,paragraph=True)
        for res in result1:
            line=line+' '+res
    '''else:
        image = cv2.imread(path+'/'+i)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Remove horizontal lines
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
        detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
        cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            cv2.drawContours(thresh, [c], -1, (0,0,0), 2)

        # Dilate to connect text and remove dots
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,1))
        dilate = cv2.dilate(thresh, kernel, iterations=2)
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 100:
                cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

        result = cv2.bitwise_and(image, image, mask=dilate)
        result[dilate==0] = (255,255,255)
        result1 = reader.readtext(result,detail=0,paragraph=True)
        line=''
        for res in result1:
            line=line+' '+res'''
    f= open('/content/sample_data/UntitledFolder1/'+str(fil)+".txt","w+")
    for i in line:
        f.write(i)
    f.close()
    print('/content/sample_data/UntitledFolder1/'+str(fil)+".txt")
    fil+=1

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


financial_document_form 16_2019_11_26-12_44_23_568541_.jpg
jpg
/content/sample_data/UntitledFolder1/1.txt
financial_document_form 16_2019_11_26-12_44_23_568244_.jpg
jpg
/content/sample_data/UntitledFolder1/2.txt
financial_document_form 16_2019_11_26-12_44_23_406781_.jpg
jpg
/content/sample_data/UntitledFolder1/3.txt
financial_document_form 16_2019_11_26-12_44_23_569090_.pdf
pdf
/content/sample_data/UntitledFolder1/4.txt
financial_document_form 16_2019_11_26-12_44_23_576851_.pdf
pdf


ValueError: ignored